# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../output_data/cities.csv")
cleaned_df = df[["City", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country", "Date"]]
cleaned_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vaini,-21.2000,-175.2000,66.36,100,20,1.14,TO,1654284469
1,Punta Arenas,-53.1500,-70.9167,41.11,60,75,16.11,CL,1654284278
2,Jamestown,42.0970,-79.2353,71.73,31,40,12.66,US,1654284470
3,Dillon,34.4166,-79.3712,90.23,38,75,6.64,US,1654284470
4,Iracoubo,5.4802,-53.2011,76.41,90,100,12.73,GF,1654284470
...,...,...,...,...,...,...,...,...,...
554,Muros,42.7762,-9.0603,66.47,77,60,6.20,ES,1654284686
555,Jiangkou,25.4872,119.1986,75.29,96,100,3.36,CN,1654284687
556,Marevo,57.3146,32.0804,49.14,91,23,5.35,RU,1654284687
557,Rubizhne,49.0123,38.3797,66.83,93,99,8.68,UA,1654284687


In [3]:
sum_stats = cleaned_df.describe()
sum_stats

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,5.590000e+02
mean,20.157488,19.516587,66.710984,68.692308,57.291592,8.517335,1.654285e+09
std,33.304694,89.585637,16.015406,22.837555,40.230774,5.294975,1.321469e+02
min,-54.800000,-175.200000,25.360000,1.000000,0.000000,0.000000,1.654284e+09
25%,-7.776800,-57.369300,54.820000,56.500000,15.500000,4.610000,1.654284e+09
50%,22.566700,23.609700,68.450000,75.000000,75.000000,7.140000,1.654285e+09
75%,48.325600,100.366750,78.900000,86.000000,100.000000,11.630000,1.654285e+09
max,78.218600,177.483300,105.690000,100.000000,100.000000,25.280000,1.654285e+09


In [4]:
#to use to center map
mean_lat = sum_stats.iloc[1,0]
mean_lng = sum_stats.iloc[1,1]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [6]:
locations = cleaned_df[["Lat","Lng"]]
humidity = cleaned_df["Humidity"]

In [7]:
# Plot Heatmap
fig = gmaps.figure(center=[mean_lat,mean_lng] ,zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#ideal weather conditions: max temp 70-80, humidity less than 80, wind speed less than 10mph, cloudiness less than 5, 

narrowed_down_df = cleaned_df.loc[(cleaned_df["Max Temp"] >= 70) & (cleaned_df["Max Temp"] <= 80) & (cleaned_df["Humidity"] < 80) & (cleaned_df["Wind Speed"] < 9) & (cleaned_df["Cloudiness"] < 5)]
narrowed_down_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,Bethel,41.3712,-73.4140,75.92,62,0,4.61,US,1654284471
28,Kapaa,22.0752,-159.3190,77.56,78,0,8.05,US,1654284476
53,Albany,42.6001,-73.9662,76.41,40,2,1.99,US,1654284418
78,Nekā,36.6508,53.2990,72.07,40,0,3.40,IR,1654284491
83,Mitsamiouli,-11.3847,43.2844,79.05,72,1,8.14,KM,1654284492
112,Amposta,40.7131,0.5810,74.23,69,0,5.50,ES,1654284501
187,Kiryat Gat,31.6100,34.7642,70.16,58,0,4.90,IL,1654284528
204,Shahrud,36.4182,54.9763,71.04,29,0,7.36,IR,1654284534
291,Springfield,37.2153,-93.2982,76.68,45,0,5.75,US,1654284026
338,Amberley,39.2048,-84.4280,78.82,31,0,4.61,US,1654284587


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = narrowed_down_df
hotel_df.loc[:,'Hotel Name'] = ''
hotel_df

/Users/mollybruns/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,Bethel,41.3712,-73.4140,75.92,62,0,4.61,US,1654284471,
28,Kapaa,22.0752,-159.3190,77.56,78,0,8.05,US,1654284476,
53,Albany,42.6001,-73.9662,76.41,40,2,1.99,US,1654284418,
78,Nekā,36.6508,53.2990,72.07,40,0,3.40,IR,1654284491,
83,Mitsamiouli,-11.3847,43.2844,79.05,72,1,8.14,KM,1654284492,
112,Amposta,40.7131,0.5810,74.23,69,0,5.50,ES,1654284501,
187,Kiryat Gat,31.6100,34.7642,70.16,58,0,4.90,IL,1654284528,
204,Shahrud,36.4182,54.9763,71.04,29,0,7.36,IR,1654284534,
291,Springfield,37.2153,-93.2982,76.68,45,0,5.75,US,1654284026,
338,Amberley,39.2048,-84.4280,78.82,31,0,4.61,US,1654284587,


In [10]:
# params dictionary to update each iteration

params = {
    "radius": 5000,
    #"types": "hotel",
    "keyword": "hotel",
    "key": g_key}

In [11]:
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    r = requests.get(base_url, params=params).json()
    #in case data is missing
    try:
        hotel_df.loc[index, "Hotel Name"] = r["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#not populating a result for albany - need to fix

/Users/mollybruns/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing field/result... skipping.


In [12]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,Bethel,41.3712,-73.4140,75.92,62,0,4.61,US,1654284471,Hampton Inn Danbury
28,Kapaa,22.0752,-159.3190,77.56,78,0,8.05,US,1654284476,The ISO
53,Albany,42.6001,-73.9662,76.41,40,2,1.99,US,1654284418,
78,Nekā,36.6508,53.2990,72.07,40,0,3.40,IR,1654284491,Hotel Milad Neka
83,Mitsamiouli,-11.3847,43.2844,79.05,72,1,8.14,KM,1654284492,Al Camar Lodge
112,Amposta,40.7131,0.5810,74.23,69,0,5.50,ES,1654284501,Hotel Hcc Montsià
187,Kiryat Gat,31.6100,34.7642,70.16,58,0,4.90,IL,1654284528,Desert Gat
204,Shahrud,36.4182,54.9763,71.04,29,0,7.36,IR,1654284534,Royal Hotel
291,Springfield,37.2153,-93.2982,76.68,45,0,5.75,US,1654284026,University Plaza Hotel & Convention Center
338,Amberley,39.2048,-84.4280,78.82,31,0,4.61,US,1654284587,Holiday Inn Express & Suites Cincinnati NE - R...


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))